<a href="https://colab.research.google.com/github/olaviinha/MusicWithChatGPT/blob/main/abc2midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">abc2mid<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">ABC to MIDI converter</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/MusicWithChatGPT" target="_blank"><font color="#999" size="4">Github</font></a>

Converts ABC notations (`.abc` files) to MIDI (`.mid` files).

- Paths should be relative to your Google Drive root (My Drive).<br>E.g. `input` value should be `Music/ABC-files/rasputin.abc` if you have a directory called _Music_ in your drive, containing a subdirectory called _ABC-files_, containing a file _rasputin.abc_.
- Should you opt to not mount Drive, use the _faux_drive_ directory (`/content/faux_drive/`) found in the Files browser on the left side bar to upload/download files, and enter paths relative to that directory.<br>E.g. `input` value should be `rasputin.abc` if you uploaded `rasputin.abc` in the _faux_drive_ directory.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>

force_setup = False
repositories = ['https://github.com/sshlien/abcmidi']
pip_packages = ''
apt_packages = ''
mount_drive = True #@param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive == True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e ./{install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import time, sys
from datetime import timedelta
import math

sys.path.append('/content/ly2midi')

# DO stuff

# Build abc2midi
%cd /content/abcmidi
!make
!make install
%cd /content

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.', time=True)

In [ ]:
#@title # Convert `.abc` to`.mid`
input = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
end_session_when_done = False #@ param {type: "boolean"}

uniq_id = gen_id()

if os.path.isfile(drive_root+input):
  inputs = [drive_root+input]
  dir_in = path_dir(drive_root+input)
elif input != '' and os.path.isdir(drive_root+input):
  dir_in = drive_root+fix_path(input)
  # What to do if input is directory path
  inputs = glob(dir_in+'/*.abc')
elif os.path.isdir(drive_root+input) and '*' in input:
  dir_in = path_dir(drive_root+input)
  inputs = glob(drive_root+input)
else:
  op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
  sys.exit('Input not understood.')

# Output
if output_dir == '':
  if mount_drive is True:
    dir_out = dir_in
  else:
    dir_out = drive_root
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()
total = len(inputs)

# -- DO THINGS --
for i, input in enumerate(inputs, 1):
  ndx_info = str(i)+'/'+str(total)+' '
  op(c.title, ndx_info+'Processing', input, time=True)
  file_out = dir_out+basename(input)+'__'+uniq_id+str(i).zfill(2)+'.mid'
  !abc2midi "{input}" -o "{file_out}"
  if os.path.isfile(file_out):
    op(c.ok, 'File saved as', file_out.replace(drive_root, ''), time=True)
  else:
    op(c.fail, 'ERROR saving file', file_out.replace(drive_root, ''), time=True)
# -- END THINGS --

timer_end = time.time()

print()
op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
op(c.ok, 'FIN.')

if end_session_when_done is True: end_session()